<a href="https://colab.research.google.com/github/profteachkids/CHE5136_Fall2021/blob/main/Secant_Newton_inclass.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
import numpy as np
import jax
import jax.numpy as jnp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'
eps= np.finfo(np.float64).eps

In [49]:
def f(x):
    return (x-3.)*(x+1.)*(x+3.)*(x-6.)

In [50]:
x=np.linspace(-4,7,100)
fig=make_subplots()
fig.add_scatter(x=x,y=f(x), mode='lines')
fig.update_layout(width=800,height=600)

In [51]:


x1, x2 = 2., 7. 
xlist = []
xlist.append(x1)
xlist.append(x2)
for i in range(100):
    y1,y2=f(x1),f(x2)
    m = (y2-y1)/(x2-x1 + eps)
    x3 = x2 - y2/(m + eps)
    xlist.append(x3)
    if np.abs(x2-x1) < 1e-10:
        break
    x1= x2
    x2=x3

xlist = np.array(xlist)
ylist = f(xlist)
labels = list(range(1, xlist.size+1))
fig2=make_subplots()
fig2.add_scatter(x=x,y=f(x), mode='lines')
fig2.add_scatter(x=xlist,y=ylist, mode='markers' , customdata= labels, hovertemplate='%{customdata}')
fig2.update_layout(width=800,height=600)

In [52]:
def der(fun, x, dx=1e-6):
    return ((fun(x+dx)- fun(x))/dx)

In [53]:
def newton(fun, x0, dx=1e-5, tol=1e-10, iter=100):
    x1=x0
    for i in range(iter):
        y1 = fun(x1)
        if np.abs(y1)<tol:
            break
        x1 = x1 - y1/der(fun, x1, dx)
    return(dict(fun=fun(x1), x=x1, iter=i))



In [54]:
def f2(x):
    x = np.atleast_1d(x)
    return np.array([x[0]  + 0.5 * (x[0] - x[1])**3 - 1.0,
            0.5 * (x[1] - x[0])**3 + x[1]])

In [55]:
def jf2(x):
    x = jnp.atleast_1d(x)
    return jnp.array([x[0]  + 0.5 * (x[0] - x[1])**3 - 1.0,
            0.5 * (x[1] - x[0])**3 + x[1]])

In [56]:
def jac(f, x0, dx=1e-5):
    N=x0.size
    J = np.zeros((N,N))
    f0=f(x0)
    for i in range(N):
        x0[i]+=dx
        J[:,i]=(f(x0)-f0)/dx
        x0[i]-=dx

    return J

In [57]:
def newton_Nd(fun, x0, dx=1e-5, tol=1e-10, iter=100):
    x1=x0
    for i in range(iter):
        y1 = fun(x1)
        if np.all(np.abs(y1)<tol):
            break
        J = jac(fun, x1)

        x1 = x1 - np.linalg.inv(J) @ y1
    return(dict(fun=fun(x1), x=x1, iter=i))

In [71]:
def broyden1(fun, x1, dx=1e-5, tol=1e-10, iter=100):
    y1 = fun(x1)
    J = jac(fun, x1)
    Jinv = np.linalg.inv(J)
    for i in range(iter):
        if np.all(np.abs(y1)<tol):
            break
        dx = - Jinv @ y1
        x2 = x1 + dx
        y2 = fun(x2)
        dy = y2-y1

        dx = dx.reshape(-1,1)
        dy = dy.reshape(-1,1)
        Jinv = Jinv + (dx - Jinv@dy)/(dx.T @ Jinv @ dy) @ dx.T @ Jinv
        y1 = y2
        x1 = x2

    return(dict(fun=fun(x1), x=x1, iter=i))

In [72]:
broyden1(f2, np.array([0.1,0.1]))

{'fun': array([-1.28486111e-12,  1.28486111e-12]),
 'iter': 8,
 'x': array([0.8411639, 0.1588361])}

In [64]:
v = np.arange(5*2)

In [65]:
v

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [68]:
v.reshape((5,-1))

array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7],
       [8, 9]])